In [1]:
import wandb
import nltk
from nltk.stem.porter import *
from torch.nn import *
from torch.optim import *
import numpy as np
import pandas as pd
import torch,torchvision
import random
from tqdm import *
from torch.utils.data import Dataset,DataLoader
stemmer = PorterStemmer()
PROJECT_NAME = 'kickstarter-NLP-V4'
device = 'cuda'

In [2]:
def tokenize(sentecne):
    return nltk.word_tokenize(sentecne)

In [3]:
tokenize('$100')

['$', '100']

In [4]:
def stem(word):
    return stemmer.stem(word.lower())

In [5]:
stem('organic')

'organ'

In [6]:
def bag_of_words(tokenized_words,all_words):
    tokenized_words = [stem(w) for w in tokenized_words]
    bag = np.zeros(len(all_words))
    for idx,w in enumerate(all_words):
        if w in tokenized_words:
            bag[idx] = 1.0
    return bag

In [7]:
bag_of_words(['hi'],['hi','how','hi'])

array([1., 0., 1.])

In [8]:
data = pd.read_csv('./data.csv')
X = data['']
y = data['']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [9]:
data = pd.read_csv('./data.csv')

In [10]:
data

         id keyword location  \
0         1     NaN      NaN   
1         4     NaN      NaN   
2         5     NaN      NaN   
3         6     NaN      NaN   
4         7     NaN      NaN   
...     ...     ...      ...   
7608  10869     NaN      NaN   
7609  10870     NaN      NaN   
7610  10871     NaN      NaN   
7611  10872     NaN      NaN   
7612  10873     NaN      NaN   

                                                   text  target  
0     Our Deeds are the Reason of this  # earthquake...       1  
1              Forest fire near La Ronge Sask .  Canada       1  
2     All residents asked to  ' shelter in place '  ...       1  
3     13,000 people receive  # wildfires evacuation ...       1  
4     Just got sent this photo from Ruby  # Alaska a...       1  
...                                                 ...     ...  
7608  Two giant cranes holding a bridge collapse int...       1  
7609   @ Aria Ahrary  @ TheTawniest The out of contr...       1  
7610  M1 . 94  [ 01 :

In [11]:
data = pd.read_csv('./data.csv').sample(frac=1)

In [12]:
data

         id              keyword               location  \
234     334          annihilated                    NaN   
2149   3084               deaths                    NaN   
6621   9484            terrorism                    NaN   
6001   8572              screams                    NaN   
6288   8983                storm  Desert Storm?? |BCHS|   
...     ...                  ...                    ...   
6980  10011              twister             Everywhere   
4221   5996            hazardous                    NaN   
5467   7801           quarantine              Bucharest   
1108   1601               bombed                    NaN   
1363   1966  burning%20buildings          Selma2Oakland   

                                                   text  target  
234    @ TomcatArts thus explaining why you were all...       1  
2149                   It is Time To Reduce Gun Deaths        1  
6621   @ Kurt Schlichter  He is already done it by n...       1  
6001   @ heyot6 Im not home

In [13]:
data = pd.read_csv('./data.csv').sample(frac=1)[:5000].sample(frac=1)

In [14]:
data

        id               keyword                    location  \
5776  8243               rioting                      Dublin   
87     129              accident                    Maldives   
3222  4624  emergency%20services                         USA   
763   1104             blew%20up                         NaN   
1370  1975          bush%20fires              Queen Creek AZ   
...    ...                   ...                         ...   
6083  8690              sinkhole                         NaN   
5125  7310     nuclear%20reactor                       Pluto   
5565  7940             rainstorm                 Fairfax, VA   
5370  7662                 panic  Charlotte, NC | KÌ¦ln, NRW   
3279  4705             epicentre                         CLT   

                                                   text  target  
5776  New doco tonight at 9pm Setanta Sports Ireland...       0  
87    RT nAAYf :  First accident in years .  Turning...       1  
3222   # Breaking  # News  -  Cal

In [15]:
X = data['text']
y = data['target']
words = []
data = []
labels = {}
labels_r = {}
idx = 0

In [16]:
for label in tqdm(y):
    if label not in list(labels.keys()):
        idx += 1
        labels[label] = idx
        labels_r[idx] = label

In [17]:
for X_batch,y_batch in tqdm(zip(X,y)):
    X_batch = tokenize(X_batch)
    new_X = []
    for Xb in X_batch:
        new_X.append(stem(Xb))
    words.extend(new_X)
    data.append([
        new_X,
        np.eye(labels[y_batch],len(labels))[labels[y_batch]-1]
    ])

In [18]:
words = sorted(set(words))
np.random.shuffle(words)
np.random.shuffle(data)
np.random.shuffle(words)
np.random.shuffle(data)

In [19]:
X = []
y = []

In [20]:
for sentence,tag in tqdm(data):
    X.append(bag_of_words(sentence,words))
    y.append(tag)

In [21]:
from sklearn.model_selection import *
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.125,shuffle=False)
X_train = torch.from_numpy(np.array(X_train)).to(device).float()
y_train = torch.from_numpy(np.array(y_train)).to(device).float()
X_test = torch.from_numpy(np.array(X_test)).to(device).float()
y_test = torch.from_numpy(np.array(y_test)).to(device).float()

In [22]:
def get_loss(model,X,y,criterion):
    preds = model(X)
    loss = criterion(preds,y)
    return loss.item()

In [23]:
def get_accuracy(model,X,y):
    preds = model(X)
    correct = 0
    total = 0
    for pred,yb in tqdm(zip(preds,y)):
        pred = int(torch.argmax(pred))
        yb = int(torch.argmax(yb))
        if pred == yb:
            correct += 1
        total += 1
    acc = round(correct/total,3)*100
    return acc

In [24]:
class Model(Module):
    def __init__(self):
        super().__init__()
        self.activation = ReLU()
        self.linear1 = Linear(len(words),512)
        self.linear2 = Linear(512,512)
        self.linear3 = Linear(512,512)
        self.linear4 = Linear(512,512)
        self.linear5 = Linear(512,512)
        self.output = Linear(512,len(labels))
    
    def forward(self,X):
        preds = self.linear1(X)
        preds = self.activation(self.linear2(preds))
        preds = self.activation(self.linear3(preds))
        preds = self.activation(self.linear4(preds))
        preds = self.activation(self.linear5(preds))
        preds = self.output(preds)
        return preds

In [25]:
model = Model().to(device)
criterion = MSELoss()
optimizer = Adam(model.parameters(),lr=0.001)
epochs = 100
batch_size = 32

In [26]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizers.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()

In [27]:
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(epochs)):
    for i in range(0,len(X_train),batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        preds = model(X_batch)
        loss = criterion(preds,y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    model.eval()
    torch.cuda.empty_cache()
    wandb.log({'Loss':(get_loss(model,X_train,y_train,criterion)+get_loss(model,X_batch,y_batch,criterion)/2)})
    torch.cuda.empty_cache()
    wandb.log({'Val Loss':get_loss(model,X_test,y_test,criterion)})
    torch.cuda.empty_cache()
    wandb.log({'Acc':(get_accuracy(model,X_train,y_train)+get_accuracy(model,X_batch,y_batch))/2})
    torch.cuda.empty_cache()
    wandb.log({'Val Acc':get_accuracy(model,X_test,y_test)})
    torch.cuda.empty_cache()
    model.train()
wandb.finish()
torch.cuda.empty_cache()